In [18]:
# Install necessary packages
!pip install pandas numpy scikit-learn imbalanced-learn gensim contractions emoji nltk tqdm joblib

import pandas as pd
import numpy as np
import re
import contractions
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import os
import joblib
from tqdm import tqdm
import warnings
from gensim.models import KeyedVectors
import gdown
import logging

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

STOP_WORDS = set(stopwords.words('english'))
WNL = WordNetLemmatizer()

# Preprocessing functions
def clean_text(text: str) -> str:
    """Clean raw text by removing URLs, emojis, mentions, and MBTI codes."""
    text = text.lower()
    text = re.sub(r'https?\S+|www\S+', '', text)
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'@\w+|#', '', text)
    text = re.sub(r"[^a-z\']", ' ', text)
    # Remove MBTI type codes
    text = re.sub(r'\b(I|E)(N|S)(F|T)(J|P)(S)?\b', '', text, flags=re.IGNORECASE)
    # Remove common footer
    text = re.sub(r'\bsent (from )?my \w+(\s\w+)? using tapatalk\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'w w w', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_posts(posts_str: str) -> str:
    """Split multi-post string (separated by '|||'), clean each, and join."""
    posts = posts_str.split('|||')
    cleaned_posts = [clean_text(post) for post in posts]
    joined = ' '.join(cleaned_posts)
    return re.sub(r'\s+', ' ', joined).strip()

def tokens_with_lemma(text: str):
    """Tokenize with lemmatization."""
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    lemmatized = [WNL.lemmatize(word.lower()) for word in filtered_tokens]
    return lemmatized

def encode_mbti_type(mbti: str):
    """Encode MBTI type as 4 binary dimensions."""
    return (
        1 if mbti[0] == 'I' else 0,
        1 if mbti[1] == 'N' else 0,
        1 if mbti[2] == 'F' else 0,
        1 if mbti[3] == 'J' else 0,
    )

# Download pretrained Word2Vec if not present
def download_pretrained_w2v():
    file_id = '0B7XkCwpI5KDYNlNUTTlSS21pQmM'  # Google News vectors
    destination = 'GoogleNews-vectors-negative300.bin.gz'
    if not os.path.exists(destination):
        gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)
    if not os.path.exists('GoogleNews-vectors-negative300.bin'):
        !gunzip GoogleNews-vectors-negative300.bin.gz
    return 'GoogleNews-vectors-negative300.bin'

# Function to average word embeddings
def average_embedding(tokens, model, embedding_size=300):
    valid_embeddings = []
    for token in tokens:
        if token in model:
            valid_embeddings.append(model[token])
    if not valid_embeddings:
        return np.zeros(embedding_size)
    return np.mean(valid_embeddings, axis=0)

# Prepare embeddings
def prepare_embeddings(df, model, embedding_size=300):
    embeddings = []
    for tokens in tqdm(df['tokens']):
        emb = average_embedding(tokens, model, embedding_size)
        embeddings.append(emb)
    return np.array(embeddings)

# Main function
def main():
    # Load raw data
    logger.info("📂 Loading raw data...")
    df = pd.read_csv('mbti_1.csv')  # Adjust path if needed, e.g., '/content/mbti_1.csv' in Colab

    # Preprocess
    logger.info("🧹 Preprocessing posts...")
    df['cleaned_posts'] = df['posts'].apply(preprocess_posts)
    df['tokens'] = df['cleaned_posts'].apply(tokens_with_lemma)

    # Encode MBTI
    logger.info("🔢 Encoding MBTI types...")
    df[['IE', 'NS', 'FT', 'JP']] = df.apply(
        lambda row: pd.Series(encode_mbti_type(row['type'])), axis=1
    )

    # Split data
    train_df, test_df = train_test_split(
        df, test_size=0.2, random_state=42, stratify=df['type']
    )

    # Download and load pretrained Word2Vec
    w2v_path = download_pretrained_w2v()
    logger.info("🧠 Loading pretrained Word2Vec...")
    model = KeyedVectors.load_word2vec_format(w2v_path, binary=True)

    # Generate embeddings
    logger.info("📝 Generating embeddings for train...")
    X_train_emb = prepare_embeddings(train_df, model)

    logger.info("📝 Generating embeddings for test...")
    X_test_emb = prepare_embeddings(test_df, model)

    # Results storage
    results_binary = {dim: {} for dim in ['IE', 'NS', 'FT', 'JP']}
    results_binary_balanced = {dim: {} for dim in ['IE', 'NS', 'FT', 'JP']}

    # Train and evaluate
    for dim in ['IE', 'NS', 'FT', 'JP']:
        logger.info(f"🧠 Training binary model for {dim} with SMOTE...")
        y_train_bin = train_df[dim]
        y_test_bin = test_df[dim]

        smote = SMOTE(random_state=42)
        X_train_res, y_train_res = smote.fit_resample(X_train_emb, y_train_bin)

        clf = LogisticRegression(max_iter=1000)
        clf.fit(X_train_res, y_train_res)

        y_pred_bin = clf.predict(X_test_emb)
        acc_bin = accuracy_score(y_test_bin, y_pred_bin)
        f1_bin = f1_score(y_test_bin, y_pred_bin, average='weighted')
        results_binary[dim] = {'accuracy': acc_bin, 'f1': f1_bin}

        print(f"{dim} - Accuracy: {acc_bin:.4f}, F1: {f1_bin:.4f}")
        print(classification_report(y_test_bin, y_pred_bin))

        # Save model (optional in Colab)
        clf_path = f'binary_{dim}_pretrained_w2v.pkl'
        joblib.dump(clf, clf_path)

    # Balanced test evaluation
    logger.info("\n🔍 Evaluating on SMOTE-balanced test...")
    for dim in ['IE', 'NS', 'FT', 'JP']:
        y_test_bin = test_df[dim]
        clf = joblib.load(f'binary_{dim}_pretrained_w2v.pkl')

        smote_test = SMOTE(random_state=42)
        X_test_res, y_test_res = smote_test.fit_resample(X_test_emb, y_test_bin)

        y_pred_bin = clf.predict(X_test_res)

        acc_bin = accuracy_score(y_test_res, y_pred_bin)
        f1_bin = f1_score(y_test_res, y_pred_bin, average='weighted')

        results_binary_balanced[dim] = {'accuracy': acc_bin, 'f1': f1_bin}
        print(f"{dim} - Accuracy (on SMOTE-balanced test): {acc_bin:.4f}, F1: {f1_bin:.4f}")
        print(classification_report(y_test_res, y_pred_bin))

    # Compare results
    for dim in ['IE', 'NS', 'FT', 'JP']:
        print(f"\n📊 Binary {dim} Results:")
        print(f"Original Test: Accuracy={results_binary[dim]['accuracy']:.4f}, F1={results_binary[dim]['f1']:.4f}")
        print(f"Balanced Test: Accuracy={results_binary_balanced[dim]['accuracy']:.4f}, F1={results_binary_balanced[dim]['f1']:.4f}")

    logger.info("✅ Process complete!")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=bcb8d786-8faa-4e17-af25-6492621649b1
To: /content/GoogleNews-vectors-negative300.bin.gz
100%|██████████| 1.65G/1.65G [00:21<00:00, 76.4MB/s]
100%|██████████| 1735/1735 [00:03<00:00, 513.47it/s]


IE - Accuracy: 0.6824, F1: 0.7046
              precision    recall  f1-score   support

           0       0.39      0.65      0.49       401
           1       0.87      0.69      0.77      1334

    accuracy                           0.68      1735
   macro avg       0.63      0.67      0.63      1735
weighted avg       0.76      0.68      0.70      1735

NS - Accuracy: 0.6692, F1: 0.7191
              precision    recall  f1-score   support

           0       0.24      0.66      0.36       240
           1       0.92      0.67      0.78      1495

    accuracy                           0.67      1735
   macro avg       0.58      0.66      0.57      1735
weighted avg       0.83      0.67      0.72      1735

FT - Accuracy: 0.7418, F1: 0.7422
              precision    recall  f1-score   support

           0       0.71      0.75      0.73       796
           1       0.77      0.74      0.76       939

    accuracy                           0.74      1735
   macro avg       0.74   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')